In [ ]:
#Image detection for YOLOv3

In [2]:
import cv2
import numpy as np

Weights ="assets/mask-yolov3-tiny-prn.weights"
cfg = "assets/mask-yolov3-tiny-prn.cfg"

net = cv2.dnn.readNetFromDarknet(cfg, Weights)
classes = []
with open("assets/classes.names", 'r') as f:
    classes = f.read().splitlines()

img= cv2.imread("assets/Mask_4.jpg")
#img = cv2.resize(orig, (960, 540))  
height, width, _ = img.shape

blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB=True, crop=False)
net.setInput(blob)
output_layers_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_layers_names)

boxes = []
confidences = []
class_ids = []

for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            #print(class_id,detection[0],detection[1],detection[2],detection[3])
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

            
font = cv2.FONT_HERSHEY_PLAIN
colors = [(0, 255, 0), (0, 165, 255), (0, 0, 255)]

if len(indexes) > 0:
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        
        # draw a bounding box rectangle and label on the image
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        text = "%s (%s)" % (label, round(confidences[i], 2))
        cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
        
        info_loc_w = w//7
        info_loc_h = h//7
        tt = 2 #text thickness
        tfs = 3 #text font size
        
        if (label == 'improper_mask'):
            img = cv2.putText(img, 'Warning: Incorrect Facemask Detected', (info_loc_w, info_loc_h), font, tfs, (0, 0, 255), tt)
        elif (label == 'no_mask'):
            img = cv2.putText(img, 'Warning: No Facemask Detected',(info_loc_w, info_loc_h), font, tfs, (0, 0, 255), tt)
        else:
            img = cv2.putText(img, 'All good',(info_loc_w, info_loc_h), font, tfs, (0, 0, 255), tt)
            
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

